In [273]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/advertising-cmaes/advertising.csv


In [274]:
!pip install pyswarms

In [275]:
import pandas as pd
advertising = pd.read_csv("../input/advertising-cmaes/advertising.csv")

In [276]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

import csv 
import matplotlib.pyplot as plt

import pickle
import random
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
import h5py
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [277]:
print(advertising.head(5))

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

                           Ad Topic Line            City  Male     Country  \
0     Cloned 5thgeneration orchestration     Wrightburgh     0     Tunisia   
1     Monitored national standardization       West Jodi     1       Nauru   
2       Organic bottom-line service-desk        Davidton     0  San Marino   
3  Triple-buffered reciprocal time-frame  West Terrifurt     1       Italy   
4          Robust logistical utilization    South Manuel     0     Iceland   

             Timestamp  Clicked on Ad  
0  2016-03-27 00:53:11              0  
1  2016-04-04 01:39:02  

In [278]:
advertising.drop(['City', 'Country','Timestamp'], axis = 1, inplace=True) 

In [279]:
uni = advertising['Ad Topic Line'].unique()
dictionary = {}
for i in range(len(uni)):
    dictionary[uni[i]] = i
print(advertising['Ad Topic Line'].head(20))

0                Cloned 5thgeneration orchestration
1                Monitored national standardization
2                  Organic bottom-line service-desk
3             Triple-buffered reciprocal time-frame
4                     Robust logistical utilization
5                   Sharable client-driven software
6                        Enhanced dedicated support
7                          Reactive local challenge
8                    Configurable coherent function
9                Mandatory homogeneous architecture
10                   Centralized neutral neural-net
11    Team-oriented grid-enabled Local Area Network
12            Centralized content-based focus group
13                 Synergistic fresh-thinking array
14                    Grass-roots coherent extranet
15               Persistent demand-driven interface
16               Customizable multi-tasking website
17                       Intuitive dynamic attitude
18     Grass-roots solution-oriented conglomeration
19          

In [280]:
advertising['Ad Topic Line'].replace(dictionary, inplace=True)
advertising.head(10)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,Male,Clicked on Ad
0,68.95,35,61833.90,256.09,0,0,0
1,80.23,31,68441.85,193.77,1,1,0
2,69.47,26,59785.94,236.50,2,0,0
3,74.15,29,54806.18,245.89,3,1,0
4,68.37,35,73889.99,225.58,4,0,0
5,59.99,23,59761.56,226.74,5,1,0
6,88.91,33,53852.85,208.36,6,0,0
7,66.00,48,24593.33,131.76,7,1,1
8,74.53,30,68862.00,221.51,8,1,0
9,69.88,20,55642.32,183.82,9,1,0


In [281]:
data = advertising.values
X = data[:,0:6]
Y = data[:,6]
#print(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y)
print(X_train.shape)
print(y_train.shape)

(750, 6)
(750,)


In [282]:
model = Sequential()
model.add(Dense(12, activation='sigmoid', input_shape=(6,)))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 12)                84        
_________________________________________________________________
dense_34 (Dense)             (None, 2)                 26        
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________


In [283]:
import pyswarms as ps


In [284]:
#print(y_train)
y_train = y_train.astype(int)
print(y_train)
print(y_train.shape)

[0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 0 1
 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1
 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1
 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0
 1 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0
 1 1 1 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 0 1 1
 0 1 1 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1
 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1
 1 0 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1
 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1
 1 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1
 0 1 1 0 1 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 1
 1 0 1 0 1 0 0 1 1 0 1 1 

In [285]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_squared_error
# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
    # Neural network architecture
    n_inputs = 6
    n_hidden = 12
    n_classes = 2
    
    # Roll-back the weights and biases
    W1 = params[0:72].reshape((n_inputs,n_hidden))
    b1 = params[72:84].reshape((n_hidden,))
    W2 = params[84:108].reshape((n_hidden,n_classes))
    b2 = params[108:110].reshape((n_classes,))

    # Perform forward propagation
    z1 = X_train.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # Compute for the negative log likelihood
    N = 750 # Number of samples
    corect_logprobs = -np.log(probs[range(N), y_train])
    loss = np.sum(corect_logprobs) / N

    return loss

In [286]:
def fitFunc(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    print(x.shape)
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [287]:
# Initialize swarm
options = {'c1': 0.7, 'c2': 0.8, 'w':0.9}

# Call instance of PSO
dimensions = (6 * 12) + (12 * 2) + 12 + 2
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fitFunc, iters=10)

2020-03-10 19:02:05,346 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.7, 'c2': 0.8, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/10, best_cost=0.693

(75, 110)
(75, 110)
(75, 110)


pyswarms.single.global_best:  30%|███       |3/10, best_cost=0.693

(75, 110)


pyswarms.single.global_best:  30%|███       |3/10, best_cost=0.693

...

pyswarms.single.global_best:  60%|██████    |6/10, best_cost=0.677

(75, 110)
(75, 110)
(75, 110)
(75, 110)


pyswarms.single.global_best:  90%|█████████ |9/10, best_cost=0.677

(75, 110)


pyswarms.single.global_best: 100%|██████████|10/10, best_cost=0.677
2020-03-10 19:02:05,756 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.6773794441170847, best pos: [ 3.67518905e-01  1.24388013e+00  5.86086146e-01  1.16653050e+00
  1.01254147e+00  6.44286329e-01  6.80656947e-01  1.05716472e+00
  5.10328704e-02  1.44221343e+00  7.19473928e-01  1.32596669e+00
  9.59527703e-01  1.24442997e+00  5.81882632e-01  1.29989477e+00
  4.76610327e-01  5.11923739e-01  9.02640913e-01  1.46216240e+00
  5.24099946e-01  6.62618376e-01  1.11181468e+00  2.29355335e-01
  1.01351552e+00  1.49568459e+00 -1.76423520e-02  4.80821368e-01
 -2.21047585e-02  1.98891953e+00  4.59062991e-01  9.59708020e-01
  1.37124028e+00  4.64912196e-02  1.76586351e-01  3.31411809e-01
  1.20670825e+00  1.50221921e+00 -6.96797037e-02  1.05483441e+00
  1.64998077e+00  1.86941709e-01  1.04984927e+00  2.00422496e+00
  1.28998928e+00  5.13774817e-01  2.30661486e-01  5.86126933e-01
  1.21804135e+00  5.5350

In [288]:
def predict(X, pos):
    """
    Use the trained weights to perform class predictions.
    Inputs
    ------
    X: numpy.ndarray
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    # Neural network architecture
    n_inputs = 6
    n_hidden = 12
    n_classes = 2
    
    # Roll-back the weights and biases
    W1 = pos[0:72].reshape((n_inputs,n_hidden))
    b1 = pos[72:84].reshape((n_hidden,))
    W2 = pos[84:108].reshape((n_hidden,n_classes))
    b2 = pos[108:110].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [289]:
print("Prediction accuracy :",str((predict(X_test,pos) == y_test).mean()))

Prediction accuracy : 0.456
